https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 258 kB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 11.9 MB/s 
     |████████████████████████████████| 164 kB 72.6 MB/s 
     |████████████████████████████████| 75 kB 5.5 MB/s 
     |████████████████████████████████| 141 kB 70.6 MB/s 
     |████████████████████████████████| 111 kB 77.4 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 1.8 MB 7.8 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
     |████████████████████████████████| 2.9 MB 50.2 MB/s 
     |████████████████████████████████| 126 kB 8.0 MB/s 
Cloning into 'arabert'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 544 (delta 175), reused 237 (delta 87), pack-reused 214
Receiving

In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [4]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']


# Trainer

In [5]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [6]:
dataset_name = 'NEWS'
model_name = 'aubmindlab/araelectra-base-discriminator'
task_name = 'classification'
max_len = 500

In [7]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [8]:
df_val.text

0       مصدر الصورة PA يقولون في الأمثال " في التأني ا...
1       قال عضو المكتب السياسي لحركة المقاومة الإسلامي...
2       مصدر الصورة Reuters قال المحامي والناشط الحقوق...
3       تنكر الممثل الأمريكي أرنولد شوارزنيجر بشارب وش...
4       مصدر الصورة Getty Images Image caption الصيادو...
                              ...                        
1277    إنديفور لدى عودته إلى مركز كيندي فجر اليوم ( ر...
1278    أكد المكتب الصحفي للكرملين أن الرئيس الروسي فل...
1279    اللاعب الغربي الكافر كرستيانو رونالدو حول جميع...
1280    مصدر الصورة Getty Images قالت شركة مايكروسوفت ...
1281    مصدر الصورة AFP Image caption جبهة تحرير مورو ...
Name: text, Length: 1282, dtype: object

In [9]:
class BERTDataset():
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [10]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()   ,df_val.label.to_list()  ,model_name,  max_len, label_map)

{'fake': 0, 'real': 1}


In [11]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [12]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# HyperParameter Search

In [13]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.lr_scheduler_type = 'cosine'
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000
# training_args.save_steps = 
#training_args.eval_steps = 
training_args.disable_tqdm = True
# print("Logging Step:", training_args.logging_steps)
# print("Eval Step:",training_args.eval_steps)

In [14]:
steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)

160
1280


In [15]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: 

# Regular Training

In [16]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.load_best_model_at_end=True

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [17]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: 

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.050048,0.979257,0.985591,0.973284,0.985179,23.131000,55.423000
1,No log,0.021276,0.991434,0.988195,0.994777,0.993760,23.030900,55.664000
2,No log,0.032972,0.984804,0.988207,0.981508,0.989080,23.116200,55.459000
3,0.081900,0.080008,0.976966,0.985746,0.968861,0.983619,23.204400,55.248000
4,0.081900,0.037811,0.988074,0.990914,0.985308,0.991420,23.154700,55.367000
5,0.081900,0.088346,0.978089,0.986259,0.970506,0.984399,23.073700,55.561000
6,0.000700,0.084567,0.978089,0.986259,0.970506,0.984399,23.041500,55.639000
7,0.000700,0.084220,0.978089,0.986259,0.970506,0.984399,23.125000,55.438000


              precision    recall  f1-score   support

           0       0.99      0.95      0.97       304
           1       0.98      1.00      0.99       978

    accuracy                           0.99      1282
   macro avg       0.99      0.97      0.98      1282
weighted avg       0.99      0.99      0.99      1282

[[289  15]
 [  4 974]]


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       304
           1       1.00      0.99      1.00       978

    accuracy                           0.99      1282
   macro avg       0.99      0.99      0.99      1282
weighted avg       0.99      0.99      0.99      1282

[[303   1]
 [  7 971]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       304
           1       0.99      1.00      0.99       978

    accuracy                           0.99      1282
   macro avg       0.99      0.98      0.98      1282
weighted avg       0.99      0.99      0.99      1282

[[294  10]
 [  4 974]]
              precision    recall  f1-score   support

           0       0.99      0.94      0.96       304
           1       0.98      1.00      0.99       978

    accuracy                           0.98      1282
   macro avg       0.99      0.97      0.98      1282
weighted avg       0.98     

TrainOutput(global_step=1280, training_loss=0.03227738676941953, metrics={'train_runtime': 2207.2007, 'train_samples_per_second': 0.58, 'total_flos': 16635459840336000, 'epoch': 8.0})

In [19]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.99      0.94      0.97       304
           1       0.98      1.00      0.99       978

    accuracy                           0.98      1282
   macro avg       0.99      0.97      0.98      1282
weighted avg       0.98      0.98      0.98      1282

[[287  17]
 [  3 975]]


{'epoch': 8.0,
 'eval_accuracy': 0.984399375975039,
 'eval_loss': 0.08421973139047623,
 'eval_macro_f1': 0.9780888410330034,
 'eval_macro_precision': 0.9862590378197997,
 'eval_macro_recall': 0.9705057313529222,
 'eval_runtime': 23.3256,
 'eval_samples_per_second': 54.961}

# Test

In [20]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/test_news.csv", header=0,encoding='utf-8')

In [21]:
df_test.text = df_test.text.apply(lambda x:   arabert_prep.preprocess(x))

In [22]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)


In [23]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       405
           1       0.99      1.00      0.99      1198

    accuracy                           0.99      1603
   macro avg       0.99      0.98      0.99      1603
weighted avg       0.99      0.99      0.99      1603

[[ 393   12]
 [   1 1197]]


{'epoch': 8.0,
 'eval_accuracy': 0.991890205864005,
 'eval_loss': 0.05184890702366829,
 'eval_macro_f1': 0.9891643740383831,
 'eval_macro_precision': 0.9937681853106776,
 'eval_macro_recall': 0.9847678229147344,
 'eval_runtime': 29.3408,
 'eval_samples_per_second': 54.634}

In [24]:
raw_pred, _, _ = trainer.predict(test_dataset)
raw_pred

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       405
           1       0.99      1.00      0.99      1198

    accuracy                           0.99      1603
   macro avg       0.99      0.98      0.99      1603
weighted avg       0.99      0.99      0.99      1603

[[ 393   12]
 [   1 1197]]


array([[ 4.473, -4.03 ],
       [-4.652,  5.137],
       [-4.668,  5.125],
       ...,
       [-4.68 ,  5.152],
       [-4.523,  5.016],
       [-4.668,  5.13 ]], dtype=float16)

# Predict

In [25]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [26]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [27]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[1]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[1]
[0]
[1]
[0]
[1]
[1]
Real : 9


In [28]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)